In [172]:
# Demo for synchronization of two data directories
import os
import io
import pandas as pd
import numpy as np
import farmhash
import time
import sqlite3
import json

In [ ]:
missions_readable = {   "clem1-l-spice-6-v1.0"       : "clementine",
                        "co-s_j_e_v-spice-6-v1.0"    : "cassini_orbiter",
                        "dawn-m_a-spice-6-v1.0"      : "dawn",
                        "di-c-spice-6-v1.0"          : "deep_impact",
                        "dif-c_e_x-spice-6-v1.0"     : "epoxi",
                        "ds1-a_c-spice-6-v1.0"       : "deep_space_1",
                        "grail-l-spice-6-v1.0"       : "grail",
                        "hay-a-spice-6-v1.0"         : "hayabusa",
                        "jno-j_e_ss-spice-6-v1.0"    : "juno",
                        "lro-l-spice-6-v1.0"         : "lunar_reconnaissance_orbiter",
                        "mer1-m-spice-6-v1.0"        : "mer_1",
                        "mer2-m-spice-6-v1.0"        : "mer_2",
                        "mess-e_v_h-spice-6-v1.0"    : "messenger",
                        "mex-e_m-spice-6-v1.0"       : "mars_express",
                        "mgs-m-spice-6-v1.0"         : "mars_global_surveyor",
                        "mro-m-spice-6-v1.0"         : "mars_reconnaissance_orbiter",
                        "msl-m-spice-6-v1.0"         : "mars_science_laboratory",
                        "near-a-spice-6-v1.0"        : "near",
                        "nh-j_p_ss-spice-6-v1.0"     : "new_horizons",
                        "ody-m-spice-6-v1.0"         : "mars_odyssey",
                        "ros-e_m_a_c-spice-6-v1.0"   : "rosetta",
                        "sdu-c-spice-6-v1.0"         : "stardust",
                        "vco-v-spice-6-v1.0"         : "venus_climate_orbiter",
                        "vex-e_v-spice-6-v1.0"       : "venus_express",
                        "vo1_vo2-m-spice-6-v1.0"     : "viking_orbiter"}

missions_true = {value: key for key, value in missions_readable.items()}

In [4]:
def create_dirdf(directory):
    if not os.path.exists(directory):
        print("Error: Directory '" + directory + "' does not exist.")
        return
    
    filenames = []
    hashvalues = []
    
    for root, subdir, files in os.walk(directory):
        for name in files:
            if not name[0] == ".": # ignore hidden files
                filepath = os.path.join(root, name)
                
                # hash full file contents
                # note: spice data encoding is mixed, so read as binary
                file = str(io.open(filepath,'rb').read()) 
                filenames.append(filepath.split(directory, 1)[1])
                hashvalues.append(farmhash.hash64(file))
                
    df = pd.DataFrame(data=hashvalues, index = filenames, columns = ["Hash"])
    df.index.name = directory
    return df

start = time.time()
dir1df = create_dirdf("./testdir1")
print("elapsed time: ", time.time() - start)

dir2df = create_dirdf("./testdir2")

print(dir1df, dir2df)

elapsed time:  0.0032210350036621094
                                           Hash
./testdir1                                     
/testfile1.txt             13167233149662072294
/testfile2.txt              2116770068367243914
/testfile3.txt             10117441339441774812
/testfile4.txt               407662078023551858
/testdir1A/testfile1A.txt   1855841718642996950                                            Hash
./testdir2                                     
/testfile1.txt             13167233149662072294
/testfile2.txt              2116770068367243914
/testfile3.txt             10117441339441774812
/testfile4.txt               407662078023551858
/testdir1A/testfile1A.txt   1855841718642996950


In [148]:
os.remove('./spicedb.sqlite')

In [149]:
def create_spicedb(spdir):
    if not os.path.exists(spdir):
        print("Error: Directory '" + spdir + "' does not exist.")
        return
    
    conn = sqlite3.connect('./spicedb.sqlite') # initialize db, this might move to an init func in the api
    c = conn.cursor()
    
    c.execute("CREATE TABLE SPICE (Mission TEXT)") # create missions table
    c.execute("ALTER TABLE SPICE ADD COLUMN Kernel TEXT")
    c.execute("ALTER TABLE SPICE ADD COLUMN File TEXT")
    c.execute("ALTER TABLE SPICE ADD COLUMN Path TEXT")
    c.execute("ALTER TABLE SPICE ADD COLUMN Hash TEXT") #theres probs a way to to this in one line
    c.execute("ALTER TABLE SPICE ADD COLUMN Newest INTEGER")

    # ooh spicy tabs ~ we could probably just parse first two directories from full string?????
    for mis in [m for m in os.listdir(spdir) if not m[0] == '.']:
        for ker in [k for k in os.listdir(spdir+'/'+mis) if not k[0] == '.']:
             for root, subdir, files in os.walk(spdir+'/'+mis+'/'+ker):
                for name in files:
                    if not name[0] == ".": # ignore hidden files
                        filepath = os.path.join(root, name)
                        mis_hr = missions_readable[mis]
                        fhash = farmhash.hash64(str(io.open(filepath,'rb').read())) # spice data encoding is mixed, so read as binary
                        
                        c.execute("INSERT OR IGNORE INTO SPICE (Mission, Kernel, File, Path, Hash, Newest) VALUES ('{mn}', '{kn}', '{fn}', '{fp}', '{fh}', {new})"\
                                  .format(mn=mis_hr, kn=ker, fn=name, fp=filepath, fh=fhash, new=0))
    conn.commit()

create_spicedb('./spice_data')

In [152]:
conn = sqlite3.connect('./spicedb.sqlite')
c = conn.cursor()
c.execute("SELECT * FROM SPICE WHERE Mission='clementine' AND Kernel='ck'")
# Note: fetchall() will pull the whole buffer, if you SELECT ten times, the result will be in there ten times
all_rows = c.fetchall() 
print(all_rows)

[('clementine', 'ck', 'testfile1.txt', './spice_data/clem1-l-spice-6-v1.0/ck/testfile1.txt', '13167233149662072294', 0), ('clementine', 'ck', 'testfile2.txt', './spice_data/clem1-l-spice-6-v1.0/ck/testfile2.txt', '2116770068367243914', 0), ('clementine', 'ck', 'testfile3.txt', './spice_data/clem1-l-spice-6-v1.0/ck/testfile3.txt', '10117441339441774812', 0), ('clementine', 'ck', 'testfile4.txt', './spice_data/clem1-l-spice-6-v1.0/ck/testfile4.txt', '407662078023551858', 0), ('clementine', 'ck', 'testfile1A.txt', './spice_data/clem1-l-spice-6-v1.0/ck/testdir1A/testfile1A.txt', '1855841718642996950', 0)]


In [108]:
conn.close()

In [170]:
# returns a dictionary of a single row from a sql select return
def sql_dict(sql_row):
    return { 'mission': sql_row[0],
             'kernel' : sql_row[1],
             'file'   : sql_row[2],
             'path'   : sql_row[3],
             'hash'   : sql_row[4],
             'newest' : sql_row[5] }

# returns an array of dictionaries of a whole sql select return
def sql_dict_array(sql_rows):
    dicts = []
    for row in sql_rows:
        dicts.append(sql_dict(row))
    return dicts
        

In [176]:
select_output = sql_dict_array(all_rows)
for d in select_output:
    print(json.dumps(d, indent=2))

{
  "mission": "clementine",
  "kernel": "ck",
  "file": "testfile1.txt",
  "path": "./spice_data/clem1-l-spice-6-v1.0/ck/testfile1.txt",
  "hash": "13167233149662072294",
  "newest": 0
}
{
  "mission": "clementine",
  "kernel": "ck",
  "file": "testfile2.txt",
  "path": "./spice_data/clem1-l-spice-6-v1.0/ck/testfile2.txt",
  "hash": "2116770068367243914",
  "newest": 0
}
{
  "mission": "clementine",
  "kernel": "ck",
  "file": "testfile3.txt",
  "path": "./spice_data/clem1-l-spice-6-v1.0/ck/testfile3.txt",
  "hash": "10117441339441774812",
  "newest": 0
}
{
  "mission": "clementine",
  "kernel": "ck",
  "file": "testfile4.txt",
  "path": "./spice_data/clem1-l-spice-6-v1.0/ck/testfile4.txt",
  "hash": "407662078023551858",
  "newest": 0
}
{
  "mission": "clementine",
  "kernel": "ck",
  "file": "testfile1A.txt",
  "path": "./spice_data/clem1-l-spice-6-v1.0/ck/testdir1A/testfile1A.txt",
  "hash": "1855841718642996950",
  "newest": 0
}
